In [1]:
!pip install unicodecsv

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10768 sha256=533c0ba28b49bffdf11fde4110a1838901a6e835f0a2b030b30aa55d93839120
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
Successfully built unicodecsv


In [2]:
from __future__ import print_function, division
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils.data_utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from collections import Counter
import unicodecsv
import numpy as np
import random
import sys
import os
import copy
import csv
import time
#from itertools import izip
from datetime import datetime
from math import log

Using TensorFlow backend.


In [0]:
eventlog = "/content/drive/My Drive/MSc Dissertation/Data/helpdesk.csv"

In [0]:
lines = [] #these are all the activity seq
timeseqs = [] #time sequences (differences between two events)
timeseqs2 = [] #time sequences (differences between the current and first)

In [0]:
#helper variables
lastcase = ''
line = ''
firstLine = True
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [0]:
csvfile = open(eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161
#ascii_offset = 65

In [0]:
# for row in spamreader:
#   print(row)

In [8]:
for row in spamreader: #the rows are "CaseID,ActivityID,CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        times2 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    #line+=chr(int(row[1]))
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    print(row[0],row[1],line,timediff,timediff2,timesincelastevent,"d: ",timesincelastevent.days,"x: ",timesincelastevent.seconds)
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False


Streaming output truncated to the last 5000 lines.
2917 6 ¢©§§ 1 173931 0:00:01 d:  0 x:  1
2919 1 ¢ 0 0 0:00:00 d:  0 x:  0
2919 6 ¢§ 28 28 0:00:28 d:  0 x:  28
2920 1 ¢ 0 0 0:00:00 d:  0 x:  0
2920 8 ¢© 838 838 0:13:58 d:  0 x:  838
2920 8 ¢©© 46 884 0:00:46 d:  0 x:  46
2920 2 ¢©©£ 500028 500912 5 days, 18:53:48 d:  5 x:  68028
2920 6 ¢©©£§ 4331204 4832116 50 days, 3:06:44 d:  50 x:  11204
2921 1 ¢ 0 0 0:00:00 d:  0 x:  0
2921 8 ¢© 21844 21844 6:04:04 d:  0 x:  21844
2921 9 ¢©ª 3909 25753 1:05:09 d:  0 x:  3909
2921 8 ¢©ª© 600034 625787 6 days, 22:40:34 d:  6 x:  81634
2921 6 ¢©ª©§ 86501 712288 1 day, 0:01:41 d:  1 x:  101
2921 6 ¢©ª©§§ 2 712290 0:00:02 d:  0 x:  2
2922 1 ¢ 0 0 0:00:00 d:  0 x:  0
2922 8 ¢© 123 123 0:02:03 d:  0 x:  123
2922 6 ¢©§ 62 185 0:01:02 d:  0 x:  62
2923 1 ¢ 0 0 0:00:00 d:  0 x:  0
2923 8 ¢© 17 17 0:00:17 d:  0 x:  17
2923 6 ¢©§ 85104 85121 23:38:24 d:  0 x:  85104
2924 1 ¢ 0 0 0:00:00 d:  0 x:  0
2924 1 ¢¢ 134 134 0:02:14 d:  0 x:  134
2924 8 ¢¢© 156 290 0

In [0]:
#print(timeseqs)

In [0]:
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [11]:
divisor = np.mean([item for sublist in timeseqs for item in sublist]) #average time between events
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist]) #average time between current and first events
print('divisor2: {}'.format(divisor2))

divisor: 210915.5199854121
divisor2: 409874.9012399708


In [12]:
print(lines)

['¢©§', '¢©§', '¢©§', '¢©§©§', '¢©§', '¢©§', '¤¢©§', '¢©§', '¢©§', '¢©§', '¢©©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢§', '¢©§', '¢©§§©ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¤¢©§', '¢©ª§', '¢©§§', '¢©ª§', '¢©§', '¢©ªª©§', '¢©ª§', '¢§', '¢©§', '¢©§§', '¢©ª©§', '¢©§', '¢§', '¢¢ª©ª©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢§', '§', '¢¢©§', '¢§', '¢©§', '¢©§', '¢©ªª©§', '¢©§', '¢©§', '¢©ª§', '¢©ª§©ª§', '¢¢ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§§', '¢©§', '¢©§', '¤¢©§', '¢©§', '¢¢©ª©£§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§©ª©§', '¢©§', '¤¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©ª©§', '¢¢§', '¢©§', '¢§§', '¤¢©§', '¢©ª§', '¢¢©§', '¢©§', '¢©ª©§©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©ª©ª©§', '¢©§', '¢©§', '¢©§§', '¢©§', '¤¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢¢§', '¢©§', '¢©§', '¢§', '¢©§', '¢©ª§§', '¢©§', '¢©ª©ª§', '¢¢©§', '¢©§', '¢¢©§', '

In [13]:
print(len(lines))

3804


In [14]:
print(timeseqs)

[[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551], [0, 3313, 1141], [0, 13, 1459434, 104956], [0, 3196606, 7], [0, 53, 66609], [0, 14, 315], [0, 7035, 590, 62621], [0, 1678025, 594825], [0, 181008, 26], [0, 1097476, 7958], [0, 10, 3765, 3028404], [0, 3633420, 13160], [0, 13607, 7351], [0, 192, 4062, 8447, 13], [0, 19, 1537332], [0, 8, 8520], [0, 1467145, 258095], [0, 18449, 70315], [0, 5467], [0, 96695, 412460], [0, 26, 14789, 1721036, 9, 7425, 151798, 5], [0, 30, 1241], [0, 26, 1951], [0, 31, 15], [0, 1646, 2926], [0, 1855, 5161], [0, 640, 52], [0, 423485, 3192880], [0, 25, 450951], [0, 87874, 8, 52], [0, 834, 345587, 265102], [0, 410, 20283, 1], [0, 61890, 86961, 1477477], [0, 9, 70073], [0, 83363, 1131019, 0, 80946, 504371], [0, 433641, 581149, 1208191], [0, 25], [0, 368210, 2572], [0, 10, 11050, 0], [0, 15, 86719, 493671, 8], [0, 813, 27994], [0, 100], [0, 18, 67593, 1036125, 9, 774720, 15], [0, 46, 15499], [0, 1013813, 183910], [0, 2

In [15]:
print(len(timeseqs))
print(len(timeseqs[3]))

3804
5


In [16]:
print(timeseqs2)

[[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248], [0, 3313, 4454], [0, 13, 1459447, 1564403], [0, 3196606, 3196613], [0, 53, 66662], [0, 14, 329], [0, 7035, 7625, 70246], [0, 1678025, 2272850], [0, 181008, 181034], [0, 1097476, 1105434], [0, 10, 3775, 3032179], [0, 3633420, 3646580], [0, 13607, 20958], [0, 192, 4254, 12701, 12714], [0, 19, 1537351], [0, 8, 8528], [0, 1467145, 1725240], [0, 18449, 88764], [0, 5467], [0, 96695, 509155], [0, 26, 14815, 1735851, 1735860, 1743285, 1895083, 1895088], [0, 30, 1271], [0, 26, 1977], [0, 31, 46], [0, 1646, 4572], [0, 1855, 7016], [0, 640, 692], [0, 423485, 3616365], [0, 25, 450976], [0, 87874, 87882, 87934], [0, 834, 346421, 611523], [0, 410, 20693, 20694], [0, 61890, 148851, 1626328], [0, 9, 70082], [0, 83363, 1214382, 1214382, 1295328, 1799699], [0, 433641, 1014790, 2222981], [0, 25], [0, 368210, 370782], [0, 10, 11060, 11060], [0, 15, 86734, 580405, 580413], [0, 813, 28807], [0, 100], [0

In [17]:
print(len(timeseqs2))

3804


In [0]:
# separate training data into 3 parts

elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]

fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]

fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]

In [19]:
print(fold3)

['¢©§', '¢¢©ª©§', '¢©ª§', '¢©§', '¢©§§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢§', '¢©§', '¢©ª§', '¢©ª©§', '¢©§', '¢©ª§', '¢©§', '¢©ª§', '¢©§§', '¢©§', '¢§', '¢©§', '¢©ª§', '¢©§', '¢©©ª©§', '¢©ª§', '¢©§', '¢©§', '¢¢¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢¢©§', '¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©£§', '¢©§', '¢¢¢§', '¢©ª©§', '¢©§', '¢©§§', '¢©ª§', '¢©§', '¢©§', '¢©§', '¢©§', '¤¢©ª©§', '¢©§', '¢©ª©§', '¢©ª©§', '¢©§', '¢©§§', '©ª©§§', '¢©ª§', '¢©§', '©©©§', '¢©§', '¢©§', '¢§', '¢©§©ª©§', '¢©§', '¢§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©ª§', '¢©£§', '¢¢©§§', '¢©§', '¢©§', '¢©§', '¢¢©ªª§', '¢©§', '¢¢©©§', '¢©§', '¤¢©ª©§', '¢¢©§', '¢©§', '¢©§', '¢¢©§', '¢©ª§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢§§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©£§', '¢©§', '¢©§', '¢¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©£§', '¢¢©§§', '¢©§', '¢©§', '¢¢©§', '¢©§', '¢©§', '¢©ª©§', '¢¢©§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§', '¢§',

In [0]:
#leave away fold3 for now
lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [21]:
print(lines)

['¢©§', '¢©§', '¢©§', '¢©§©§', '¢©§', '¢©§', '¤¢©§', '¢©§', '¢©§', '¢©§', '¢©©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢§', '¢©§', '¢©§§©ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¤¢©§', '¢©ª§', '¢©§§', '¢©ª§', '¢©§', '¢©ªª©§', '¢©ª§', '¢§', '¢©§', '¢©§§', '¢©ª©§', '¢©§', '¢§', '¢¢ª©ª©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢§', '§', '¢¢©§', '¢§', '¢©§', '¢©§', '¢©ªª©§', '¢©§', '¢©§', '¢©ª§', '¢©ª§©ª§', '¢¢ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§§', '¢©§', '¢©§', '¤¢©§', '¢©§', '¢¢©ª©£§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§©ª©§', '¢©§', '¤¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©ª©§', '¢¢§', '¢©§', '¢§§', '¤¢©§', '¢©ª§', '¢¢©§', '¢©§', '¢©ª©§©§', '¢©§', '¢©ª§', '¢©§', '¢©§', '¢©ª©ª©§', '¢©§', '¢©§', '¢©§§', '¢©§', '¤¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©ª©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢©§', '¢¢§', '¢©§', '¢©§', '¢§', '¢©§', '¢©ª§§', '¢©§', '¢©ª©ª§', '¢¢©§', '¢©§', '¢¢©§', '

In [22]:
print(lines[0] + '!')

¢©§!


In [23]:
lines = map(lambda x: x+'!',lines) #put delimiter symbol
lines = list(lines)
print(lines)

['¢©§!', '¢©§!', '¢©§!', '¢©§©§!', '¢©§!', '¢©§!', '¤¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©©§!', '¢©§!', '¢©§!', '¢©§!', '¢©ª§!', '¢©§!', '¢©§!', '¢©ª©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢§!', '¢©§!', '¢©§§©ª©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¤¢©§!', '¢©ª§!', '¢©§§!', '¢©ª§!', '¢©§!', '¢©ªª©§!', '¢©ª§!', '¢§!', '¢©§!', '¢©§§!', '¢©ª©§!', '¢©§!', '¢§!', '¢¢ª©ª©§!', '¢©§!', '¢©§!', '¢©§!', '¢©ª§!', '¢©§!', '¢©§!', '¢§!', '§!', '¢¢©§!', '¢§!', '¢©§!', '¢©§!', '¢©ªª©§!', '¢©§!', '¢©§!', '¢©ª§!', '¢©ª§©ª§!', '¢¢ª©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§§!', '¢©§!', '¢©§!', '¤¢©§!', '¢©§!', '¢¢©ª©£§!', '¢©§!', '¢©§!', '¢©ª©§!', '¢©§!', '¢©§©ª©§!', '¢©§!', '¤¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©§!', '¢©ª§!', '¢©§!', '¢©§!', '¢©ª©§!', '¢©§!', '¢©ª©§!', '¢¢§!', '¢©§!', '¢§§!', '¤¢©§!', '¢©ª§!', '¢¢©§!', '¢©§!', '¢©ª©§©§!', '¢©§!', '¢©ª§!', '¢©§!', '¢©§!', '¢©ª©ª©§!', '¢©§!', '¢©§!', '¢©§§!', '¢©§!', '¤¢©ª©§!', '¢©§!', '¢©§!', '¢©§!', '¢©ª©§!', '¢©

In [24]:
print(len(lines[0]))

4


In [25]:
test = map(lambda x: len(x),lines)
print(list(test))

[4, 4, 4, 6, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 6, 4, 4, 4, 4, 3, 4, 9, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 4, 7, 5, 3, 4, 5, 6, 4, 3, 8, 4, 4, 4, 5, 4, 4, 3, 2, 5, 3, 4, 4, 7, 4, 4, 5, 8, 6, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 8, 4, 4, 6, 4, 8, 4, 5, 4, 4, 4, 4, 5, 4, 4, 6, 4, 6, 4, 4, 4, 5, 5, 5, 4, 8, 4, 5, 4, 4, 8, 4, 4, 5, 4, 7, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 6, 4, 7, 5, 4, 5, 5, 4, 4, 4, 4, 4, 4, 3, 4, 5, 6, 5, 4, 4, 5, 4, 4, 8, 4, 5, 4, 3, 4, 4, 4, 4, 6, 6, 4, 4, 4, 3, 4, 3, 5, 4, 5, 4, 8, 6, 5, 4, 3, 5, 5, 3, 4, 4, 4, 3, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 5, 5, 4, 4, 4, 4, 4, 5, 3, 7, 6, 5, 4, 4, 4, 5, 4, 7, 3, 8, 5, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 5, 4, 4, 4, 8, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 6, 5, 4, 6, 4, 7, 6, 3, 5, 4, 4, 6, 6, 6, 4, 4, 4, 5, 5, 3, 4, 4, 4, 5, 4, 4, 4, 5, 5, 4, 4, 5, 4, 4, 5, 5, 6, 4, 8, 5, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 6, 5, 4, 4, 4, 6, 5, 4, 5, 4, 6, 4, 4, 5, 

In [0]:
step = 1
sentences = []
softness = 0
next_chars = []
#lines = map(lambda x: x+'!',lines) #put delimiter symbol
maxlen = max(map(lambda x: len(x),lines)) #find maximum line size

In [27]:
print(maxlen)

15


In [28]:
# next lines here to get all possible characters for events and annotate them with numbers

chars = map(lambda x: set(x),lines)
chars = list(set().union(*chars))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

total chars: 9, target chars: 10
{0: '¢', 1: '£', 2: '¤', 3: '¥', 4: '¦', 5: '§', 6: '¨', 7: '©', 8: 'ª'}


In [0]:
# lsit = [1,2,3,4,4,3,2,1]
# ls = copy.copy(lsit)
# ls

In [30]:
csvfile = open(eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers

['CaseID', 'ActivityID', 'CompleteTimestamp']

In [0]:
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []
timeseqs4 = []
times = []
times2 = []
times3 = []
times4 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [0]:
break_cnt = 0

for row in spamreader:
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
        line = ''
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    timediff3 = timesincemidnight.seconds #this leaves only time even occured after midnight
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    times.append(timediff)
    times2.append(timediff2)
    times3.append(timediff3)
    times4.append(timediff4)
    lasteventtime = t
    firstLine = False
    # print(line, timediff, timediff2, timediff3, timediff4)
    # break_cnt+=1;
    # if(break_cnt == 5): break

In [0]:
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
numlines+=1

In [0]:
elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]
fold1_t3 = timeseqs3[:elems_per_fold]
fold1_t4 = timeseqs4[:elems_per_fold]

In [35]:
with open('/content/drive/My Drive/MSc Dissertation/Data/fold1.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    break_cnt = 0
    for row, timeseq in zip(fold1, fold1_t):
        #spamwriter.writerow([unicode(s).encode("utf-8") +'#{}'.format(t) for s, t in zip(row, timeseq)])
        spamwriter.writerow([repr(str(s).encode("utf-8"))+'#{}'.format(t) for s, t in zip(row, timeseq)])
        print(row, timeseq)
        break_cnt+=1
        #if (break_cnt == 5): break


¢©§ [0, 15, 173999]
¢©§ [0, 457325, 306]
¢©§ [0, 62954, 432]
¢©§©§ [0, 2221, 109, 1660, 6746]
¢©§ [0, 2697, 64551]
¢©§ [0, 3313, 1141]
¤¢©§ [0, 13, 1459434, 104956]
¢©§ [0, 3196606, 7]
¢©§ [0, 53, 66609]
¢©§ [0, 14, 315]
¢©©§ [0, 7035, 590, 62621]
¢©§ [0, 1678025, 594825]
¢©§ [0, 181008, 26]
¢©§ [0, 1097476, 7958]
¢©ª§ [0, 10, 3765, 3028404]
¢©§ [0, 3633420, 13160]
¢©§ [0, 13607, 7351]
¢©ª©§ [0, 192, 4062, 8447, 13]
¢©§ [0, 19, 1537332]
¢©§ [0, 8, 8520]
¢©§ [0, 1467145, 258095]
¢©§ [0, 18449, 70315]
¢§ [0, 5467]
¢©§ [0, 96695, 412460]
¢©§§©ª©§ [0, 26, 14789, 1721036, 9, 7425, 151798, 5]
¢©§ [0, 30, 1241]
¢©§ [0, 26, 1951]
¢©§ [0, 31, 15]
¢©§ [0, 1646, 2926]
¢©§ [0, 1855, 5161]
¢©§ [0, 640, 52]
¢©§ [0, 423485, 3192880]
¢©§ [0, 25, 450951]
¤¢©§ [0, 87874, 8, 52]
¢©ª§ [0, 834, 345587, 265102]
¢©§§ [0, 410, 20283, 1]
¢©ª§ [0, 61890, 86961, 1477477]
¢©§ [0, 9, 70073]
¢©ªª©§ [0, 83363, 1131019, 0, 80946, 504371]
¢©ª§ [0, 433641, 581149, 1208191]
¢§ [0, 25]
¢©§ [0, 368210, 2572]
¢©§§ [0, 10, 

In [36]:
 break_cnt = 0
 for s, t in zip(row, timeseq):
    print(row, timeseq, s, t, repr(str(s).encode("utf-8"))+'#{}'.format(t))
#   #print(type(bytes(repr(str(s).encode("utf-8"))+repr('#{}'.format(t).encode("utf-8")))))
#   print(repr(str(s).encode("utf-8"))+'#{}'.format(t))
#   #print(str(s).encode("utf-8") +'#{}'.format(t))
    break_cnt+=1
    if(break_cnt == 5): break
  

¢©§ [0, 64058, 15515] ¢ 0 b'\xc2\xa2'#0
¢©§ [0, 64058, 15515] © 64058 b'\xc2\xa9'#64058
¢©§ [0, 64058, 15515] § 15515 b'\xc2\xa7'#15515


In [0]:
fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
fold2_t3 = timeseqs3[elems_per_fold:2*elems_per_fold]
fold2_t4 = timeseqs4[elems_per_fold:2*elems_per_fold]

In [0]:
with open('/content/drive/My Drive/MSc Dissertation/Data/fold2.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold2, fold2_t):
        spamwriter.writerow([repr(str(s).encode("utf-8"))+'#{}'.format(t) for s, t in zip(row, timeseq)])


In [0]:
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
fold3_t4 = timeseqs4[2*elems_per_fold:]

In [0]:
with open('/content/drive/My Drive/MSc Dissertation/Data/fold3.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold3, fold3_t):
        spamwriter.writerow([repr(str(s).encode("utf-8"))+'#{}'.format(t) for s, t in zip(row, timeseq)])


In [0]:
lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2
lines_t3 = fold1_t3 + fold2_t3
lines_t4 = fold1_t4 + fold2_t4

In [0]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = map(lambda x: x+'!',lines)

In [0]:
sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []

In [44]:
for line, line_t, line_t2, line_t3, line_t4 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4):
    for i in range(0, len(line), step):
        if i==0:
            continue

        #we add iteratively, first symbol of the line, then two first, three...

        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        next_chars.append(line[i])
        if i==len(line)-1: # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])
print('nb sequences:', len(sentences))

nb sequences: 9181


In [45]:
print(sentences)

['¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢©§©', '¢©§©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¤', '¤¢', '¤¢©', '¤¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©©', '¢©©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©ª', '¢©ª§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©ª', '¢©ª©', '¢©ª©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢©§§', '¢©§§©', '¢©§§©ª', '¢©§§©ª©', '¢©§§©ª©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¤', '¤¢', '¤¢©', '¤¢©§', '¢', '¢©', '¢©ª', '¢©ª§', '¢', '¢©', '¢©§', '¢©§§', '¢', '¢©', '¢©ª', '¢©ª§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©ª', '¢©ªª', '¢©ªª©', '¢©ªª©§', '¢', '¢©', '¢©ª', '¢©ª§', '¢', '¢§', '¢', '¢©', '¢©§', '¢', '¢©', '¢©§', '¢©§§', '¢', '¢©', '¢©ª', '¢©ª©', '¢©ª©§', '¢', '¢©', '¢©§', '¢'

In [54]:
print(next_chars)

['©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '©', '§', '!', '©', '§', '!', '©', '§', '!', '¢', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', 'ª', '§', '!', '©', '§', '!', '©', '§', '!', '©', 'ª', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '§', '!', '©', '§', '!', '©', '§', '§', '©', 'ª', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '¢', '©', '§', '!', '©', 'ª', '§', '!', '©', '§', '§', '!', '©', 'ª', '§', '!', '©', '§', '!', '©', 'ª', 'ª', '©', '§', '!', '©', 'ª', '§', '!', '§', '!', '©', '§', '!', '©', '§', '§', '!', '©', 'ª', '©', '§', '!', '©', '§', '!', '§', '!', '¢', 'ª', '©', 'ª', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', '§', '!', '©', 'ª', '§', '!', '©', '§', '!', '©', '§', '!', '§', '!', '!', '¢', '©', '§', '!', '§', '!', '©', '§', '!', '©', '§', '!', '©',

In [47]:
print(sentences_t)

[[0], [0, 15], [0, 15, 173999], [0], [0, 457325], [0, 457325, 306], [0], [0, 62954], [0, 62954, 432], [0], [0, 2221], [0, 2221, 109], [0, 2221, 109, 1660], [0, 2221, 109, 1660, 6746], [0], [0, 2697], [0, 2697, 64551], [0], [0, 3313], [0, 3313, 1141], [0], [0, 13], [0, 13, 1459434], [0, 13, 1459434, 104956], [0], [0, 3196606], [0, 3196606, 7], [0], [0, 53], [0, 53, 66609], [0], [0, 14], [0, 14, 315], [0], [0, 7035], [0, 7035, 590], [0, 7035, 590, 62621], [0], [0, 1678025], [0, 1678025, 594825], [0], [0, 181008], [0, 181008, 26], [0], [0, 1097476], [0, 1097476, 7958], [0], [0, 10], [0, 10, 3765], [0, 10, 3765, 3028404], [0], [0, 3633420], [0, 3633420, 13160], [0], [0, 13607], [0, 13607, 7351], [0], [0, 192], [0, 192, 4062], [0, 192, 4062, 8447], [0, 192, 4062, 8447, 13], [0], [0, 19], [0, 19, 1537332], [0], [0, 8], [0, 8, 8520], [0], [0, 1467145], [0, 1467145, 258095], [0], [0, 18449], [0, 18449, 70315], [0], [0, 5467], [0], [0, 96695], [0, 96695, 412460], [0], [0, 26], [0, 26, 14789], [

In [48]:
print(chars)

['¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª']


In [49]:
print('Vectorization...')
num_features = len(chars)+5
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)

Vectorization...
num features: 14


In [0]:
#print(X)

In [51]:
print(softness)
print(divisor)

0
210915.5199854121


In [52]:
print(target_chars)

['!', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª']


In [0]:
break_cnt = 0

for i, sentence in enumerate(sentences):
    leftpad = maxlen-len(sentence)
    next_t = next_chars_t[i]
    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    for t, char in enumerate(sentence):
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char: #this will encode present events to the right places
                X[i, t+leftpad, char_indices[c]] = 1
        X[i, t+leftpad, len(chars)] = t+1
        X[i, t+leftpad, len(chars)+1] = sentence_t[t]/divisor
        X[i, t+leftpad, len(chars)+2] = sentence_t2[t]/divisor2
        X[i, t+leftpad, len(chars)+3] = sentence_t3[t]/86400
        X[i, t+leftpad, len(chars)+4] = sentence_t4[t]/7
    for c in target_chars:
        if c==next_chars[i]:
            y_a[i, target_char_indices[c]] = 1-softness
        else:
            y_a[i, target_char_indices[c]] = softness/(len(target_chars)-1)
    #print(y_a[i,:],next_chars[i],target_chars)
    y_t[i] = next_t/divisor
    #np.set_printoptions(threshold=np.nan)
    np.set_printoptions(threshold=sys.maxsize)
    # print(i,sentence, next_t,y_a[i],y_t[i])
    # break_cnt+=1
    # if(break_cnt == 1): break

In [0]:
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(target_chars), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)


Build model...


In [0]:
model = Model(inputs=[main_input], outputs=[act_output, time_output])

In [0]:
opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

In [0]:
model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
#model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
model_checkpoint = ModelCheckpoint('/content/drive/My Drive/MSc Dissertation/Data/output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [0]:
model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=1, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=100)

Train on 7344 samples, validate on 1837 samples
Epoch 1/100
7344/7344 [==============================] - 68s 9ms/step - loss: 2.1585 - act_output_loss: 0.9618 - time_output_loss: 1.1966 - val_loss: 1.7714 - val_act_output_loss: 0.6832 - val_time_output_loss: 1.0833
Epoch 2/100
7344/7344 [==============================] - 67s 9ms/step - loss: 1.7671 - act_output_loss: 0.7183 - time_output_loss: 1.0503 - val_loss: 1.6274 - val_act_output_loss: 0.6223 - val_time_output_loss: 1.0005
Epoch 3/100
7344/7344 [==============================] - 67s 9ms/step - loss: 1.7147 - act_output_loss: 0.6895 - time_output_loss: 1.0255 - val_loss: 1.5705 - val_act_output_loss: 0.5896 - val_time_output_loss: 0.9758
Epoch 4/100
7344/7344 [==============================] - 67s 9ms/step - loss: 1.6899 - act_output_loss: 0.6753 - time_output_loss: 1.0136 - val_loss: 1.5717 - val_act_output_loss: 0.5861 - val_time_output_loss: 0.9806
Epoch 5/100
7344/7344 [==============================] - 66s 9ms/step - loss: 1.